This project analyzes how different Large Language Models (LLMs) respond to Retrieval-Augmented Generation (RAG). We based our implementation on the paper "RAGAS: Automated Evaluation of Retrieval Augmented Generation" by Shahul Es, Jithin James, Luis Espinosa-Anke, and Steven Schockaert. To evaluate the RAG performance of the different models, we implemented the three different scores mentioned in the paper: faithfulness, answer relevance, and context relevance. For instructions on how to run the project see the README.md file.

### Dependencies

In [ ]:
!pip install torch --index-url https://download.pytorch.org/whl/cu128 --quiet
!pip install -U bitsandbytes --quiet 
!pip install ipywidgets sentence-transformers faiss-cpu transformers datasets transformers jupyterlab_widgets pandas accelerate numpy hf_xet tqdm --quiet

In [2]:
import os
import sys
import re
#------------------------------------------------------------------------------------------------
#os.environ["HF_HOME"] = "D:/AI_Models" #-> only for my computer delete on others!!!
#------------------------------------------------------------------------------------------------
import torch
import numpy as np
from numpy.linalg import norm
import faiss
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from accelerate import Accelerator
from tqdm.notebook import tqdm

Checks if CUDA is available and sets the accelerator for hardware-agnostic execution:

In [3]:
print(f"CUDA verfügbar: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    
    accelerator = Accelerator()
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Anzahl GPUs: {torch.cuda.device_count()}")
    print(f"CUDA Version (PyTorch): {torch.version.cuda}")
    print(f"Accelerator device: {accelerator.device}")
else:
    accelerator = Accelerator(cpu=True) 
    print(f"Accelerator device: {accelerator.device}")

CUDA verfügbar: False
Accelerator device: cpu


### Import the models

Due to hardware limitations, we were unable to run all models directly one afer the other, so we always hat to comment out all models except the one we were currently using.

In [ ]:
"""
model_name_mistral = "mistralai/Mistral-7B-v0.1"
model_mistral = AutoModelForCausalLM.from_pretrained(
    model_name_mistral,
    dtype="auto"
    )
model_mistral = accelerator.prepare(model_mistral)
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name_mistral)

model_name_deepseek = "deepseek-ai/deepseek-llm-7b-base"
model_deepseek = AutoModelForCausalLM.from_pretrained(
    model_name_deepseek,
    dtype="auto"
    )
model_deepseek = accelerator.prepare(model_deepseek)
tokenizer_deepseek = AutoTokenizer.from_pretrained(model_name_deepseek)

model_name_llama = "meta-llama/Llama-2-7b"
model_llama = AutoModelForCausalLM.from_pretrained(
    model_name_llama,
    dtype="auto"
    )
model_llama = accelerator.prepare(model_llama)
tokenizer_llama = AutoTokenizer.from_pretrained(model_name_llama)
"""

#-> "Qwen/Qwen3-8B"
model_name_qwen = "Qwen/Qwen3-0.6B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16, # Use half-precision for speed
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model_qwen = AutoModelForCausalLM.from_pretrained(
    model_name_qwen,
    #quantization_config=bnb_config,
    device_map="auto",
    #attn_implementation="flash_attention_2"
)
model_qwen = accelerator.prepare(model_qwen) 

tokenizer_qwen = AutoTokenizer.from_pretrained(model_name_qwen)
tokenizer_qwen.padding_side = "left"
if tokenizer_qwen.pad_token is None:
    tokenizer_qwen.pad_token = tokenizer_qwen.eos_token


ImportError: Using `bitsandbytes` 4-bit quantization requires bitsandbytes: `pip install -U bitsandbytes>=0.46.1`

### Import the dataset

In [ ]:
rag_dataset_1200 = datasets.load_dataset("neural-bridge/rag-dataset-1200", split="all")

documents_rag_1200 = [item["context"] for item in rag_dataset_1200]
len(documents_rag_1200)

1200

In [ ]:
documents = documents_rag_1200

def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = []
for doc in documents:
    chunks.extend(chunk_text(doc))

len(chunks)

10950

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embed_model.encode(
    chunks,
    show_progress_bar=True
)

embeddings = np.array(embeddings)
embeddings.shape

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/343 [00:00<?, ?it/s]

(10950, 384)

We use the FlatL2 index to measure the similarity of the embeddings. FlatL2 ueses the euclidian distance for that.

In [ ]:
"""def retrieve_context(question, k=5):
    q_emb = embed_model.encode([question])
    distances, indices = index.search(q_emb, len(chunks))
    
    # Filter: nur Chunks mit mehr als 50 Zeichen UND Frage-Keywords enthalten
    keywords = [w.lower() for w in question.split()]
    selected_chunks = []
    for i in indices[0]:
        chunk = chunks[i].strip()
        if len(chunk) > 50 and any(kw in chunk.lower() for kw in keywords):
            selected_chunks.append(chunk)
        if len(selected_chunks) >= k:
            break
    return "\n".join(selected_chunks)"""


'def retrieve_context(question, k=5):\n    q_emb = embed_model.encode([question])\n    distances, indices = index.search(q_emb, len(chunks))\n\n    # Filter: nur Chunks mit mehr als 50 Zeichen UND Frage-Keywords enthalten\n    keywords = [w.lower() for w in question.split()]\n    selected_chunks = []\n    for i in indices[0]:\n        chunk = chunks[i].strip()\n        if len(chunk) > 50 and any(kw in chunk.lower() for kw in keywords):\n            selected_chunks.append(chunk)\n        if len(selected_chunks) >= k:\n            break\n    return "\n".join(selected_chunks)'

Prompt similar the the one from the Ragas paper

In [ ]:
def build_rag_prompt(context, question):
    return f"""Answer the question using ONLY the information from the context.
If the answer is not explicitly stated, reply with: I don't know.

Context:
{context}

Question:
{question}

Answer:
"""


In [ ]:
"""#TODO entscheide bei jedem max_tokens=1000 ob 1000 angebracht sind
def answer_without_context(model, tokenizer, prompt, max_tokens=1000, do_sample=True):
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    text_input = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True,
        #-----------------------------------------------------------
        enable_thinking=False #-> falls kein thinking model löschen!
        #-----------------------------------------------------------
    )
    inputs = tokenizer(text_input, return_tensors="pt").to(accelerator.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=do_sample,
            temperature=0.7,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_ids = output_ids[0][inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()"""

In [ ]:
def answer_without_context(model, tokenizer, questions, contexts, max_tokens=1000, do_sample=True):
    prompts = []
    for q, c in zip(questions, contexts):
        messages = [
            {"role": "user", "content": f"Context: {c}\n\nQuestion: {q}"}
        ]
        full_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        prompts.append(full_prompt)

    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=do_sample,
            temperature=0.7
        )

    generated_answers = []
    input_len = inputs.input_ids.shape[1]
    
    for i, out_ids in enumerate(output_ids):
        new_tokens = out_ids[input_len:]
        answer = tokenizer.decode(new_tokens, skip_special_tokens=True)
        generated_answers.append(answer.strip())
        
    return generated_answers

In [ ]:

def answer_with_context(model, tokenizer, prompt, context, max_tokens=1000, do_sample=True):
    rag_instruction = (
        "Answer using the information from the given context.\n"
        f"context: {context}\n\n"
        f"{prompt}"
    )

    messages = [
        {"role": "user", "content": rag_instruction}
    ]
    
    text_input = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True,
        #-----------------------------------------------------------
        enable_thinking=False #-> falls kein thinking model löschen!
        #-----------------------------------------------------------
    )
    
    inputs = tokenizer(text_input, return_tensors="pt").to(accelerator.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=do_sample,
            temperature=0.7,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )
    generated_ids = output_ids[0][inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    

### The three scores mentioned in the Ragas paper
In this section we implement all three scores mentioned in the Ragas paper.
We will use these to evaluate the models rag performance

$$F = \frac{|F|}{|V|}$$

In [ ]:
def build_statement_prompt(question: str, answer: str):
    return f"""Given a question and answer, create one or more statements from each sentence in the given answer.
question: {question}
answer: {answer}"""

In [ ]:
def extract_statements(question: str, answer: str, model, tokenizer, max_tokens=1000):
    #Extracts statements out of the model generated answer 
    prompt = build_statement_prompt(question, answer)

    answer = answer_without_context(model, tokenizer, prompt, max_tokens, do_sample=False)

    statements = [line.strip() for line in answer.split("\n") if len(line.strip()) > 3]
    return statements

Changed the prompt for better understanding

In [ ]:
def build_faithfullness_prompt(statements):
    prompt = f"""Consider the given context and following statements, the determine whether they are supported by the information presente in the context. 
Provide a brief explanation for each statement before arriving at the verdict (Yes/No). 
Answer in the format:
Statement: ...
Explanation: ...
Verdict: (Yes/No)
Do not deviate from the specified format.
These are the statements
"""
    for s in statements:
        prompt += f"Statement: {s}\n"
        
    return prompt

In [ ]:
def count_supported(answer: str):
    yes_matches = re.findall(r'Verdict:\s*(Yes)', answer, re.IGNORECASE)
    yes_count = len(yes_matches)
    return yes_count

In [ ]:
def calculate_faithfulness_score(total_statements: int, suported_statements: int):
    if suported_statements == 0.0:
        return 0.0

    return suported_statements / total_statements

$$AR = \frac{1}{n} \sum\limits^n_{i=1} sim(q,q_i)$$

In [ ]:
def build_answer_relevance_prompt(answer: str):
    prompt = f"""Generate a question for the given answer.
    answer: {answer}"""
    return prompt

In [ ]:
def extract_questions(text: str) -> list:
    text.strip()
    list = text.split("?")
    return [item + "?" for item in list]

In [ ]:
def calculate_question_similarity(original_question, generated_questions):
    q_embedding = embed_model.encode([original_question])
    gen_q_embeddings = embed_model.encode(generated_questions)

    q_embedding = np.array(q_embedding).astype('float32')
    gen_q_embeddings = np.array(gen_q_embeddings).astype('float32')

    faiss.normalize_L2(q_embedding)
    faiss.normalize_L2(gen_q_embeddings)

    d = q_embedding.shape[1]
    index = faiss.IndexFlatIP(d)
    
    index.add(gen_q_embeddings)
    
    k = len(generated_questions)
    D, I = index.search(q_embedding, k=k)
    
    scores = D[0]
    return scores

In [ ]:
#TODO
def calculate_answer_relevance_score(similatity: list):
    return float(np.mean(similatity))

$$CR = \frac{\text{number of extracted sentences}}{\text{total number of senctences in }c(q)}$$

In [ ]:
def build_context_relevance_prompt(question: str):
    prompt = f"""Please extract relevant sentences from the provided context that can potentially help ansewr the following question. If no relevant sentences are found, or if you believe the question cannot be answered from the given context, return the phares "Insufficient Information".While extracting candidate sentences you're not allowed to make any changes to senctences from the given context.
    Question: {question}"""
    return prompt

In [ ]:
#TODO vielleicht erweitern sehr simple satz zählung -> entscheiden ob auch ! und ? zählen
def count_sentences(sentence: str):
    num = sentence.count(". ")
    num += sentence.count("! ")
    num += sentence.count("? ")
    return num

In [ ]:
def calculate_context_relevance_score(num_extracted, num_sentences):
    if num_sentences == 0.0:
        return 0.0
    
    return num_extracted / num_sentences

### RAG analysis

In [ ]:
def analyse_faithfullness(model, tokenizer, context, question, answer, do_sample=True):
    #extracts statements out of the answer and generates a statements list
    statements = extract_statements(question, answer, model_qwen, tokenizer_qwen)

    #lets the model evaluate the faithfullness
    faithfullness_prompt = build_faithfullness_prompt(statements)
    #do_sample has to be false
    eval = answer_with_context(model_qwen, tokenizer_qwen, context, faithfullness_prompt, do_sample=False)
    num_supported = count_supported(eval)

    return [statements, num_supported]

In [ ]:
#TODO entscheiden wieviele fragen die funkition bearbeiten soll
def analyse_answer_relevance(model, tokenizer, question, answer, do_sample=True):
    prompt = build_answer_relevance_prompt(answer)
    questions_generated = answer_without_context(model, tokenizer, prompt, do_sample=do_sample)
    questions_generated = extract_questions(questions_generated)

    sim = calculate_question_similarity(question, questions_generated)

    return sim

In [ ]:
def analyse_context_relevance(model, tokenizer, context, question):
    prompt = build_context_relevance_prompt(question)
    answer = answer_with_context(model, tokenizer, prompt, context, do_sample=False)

    num_extracted = count_sentences(answer)
    num_sentences = count_sentences(context)
    
    return [num_extracted, num_sentences]

In [ ]:
def analyse_rag(model, tokenizer, dataset, start_index=100, num_samples=5, do_sample=True):
    scores = {
        "faithfullness score" : 0.0,
        "answer relevance score" : 0.0,
        "context relevance score" : 0.0
    }
    
    # Initialize accumulators
    statements = []
    supported_statements = 0
    similatity = []
    num_extracted = 0
    num_sentences = 0

    # --- STEP 1: PREPARE DATA BATCH ---
    # Extract the list of questions and contexts for the whole batch first
    batch_questions = []
    batch_contexts = []
    
    for i in range(num_samples):
        # Access the dataset dynamically based on start_index
        item = dataset[start_index + i] 
        batch_questions.append(item["question"])
        batch_contexts.append(item["context"])

    # --- STEP 2: BATCH INFERENCE (The Speedup) ---
    print(f"Generating {num_samples} answers in parallel...")
    # We call the model ONCE for all questions. 
    # Note: Ensure your updated answer_with_context accepts lists for questions/contexts
    batch_answers = answer_with_context(
        model, 
        tokenizer, 
        batch_questions, 
        batch_contexts, 
        do_sample=do_sample
    )

    # --- STEP 3: CALCULATE METRICS (Sequential) ---
    # We now iterate through the answers we just generated to calculate scores
    print("Calculating metrics for generated answers...")
    
    for i in tqdm(range(num_samples)):
        # Retrieve the specific data for this single iteration
        question = batch_questions[i]
        context = batch_contexts[i]
        answer = batch_answers[i] # Take the answer from our pre-computed list

        # 1. Faithfulness
        # Note: These functions are still slow if they generate text. 
        # Ideally, you would batch these too, but this requires rewriting those functions.
        faithfullness = analyse_faithfullness(model, tokenizer, context, question, answer, do_sample)
        statements.extend(faithfullness[0])
        supported_statements += faithfullness[1]

        # 2. Answer Relevance
        answer_relevance = analyse_answer_relevance(model, tokenizer, question, answer, do_sample=do_sample)
        similatity.extend(answer_relevance)

        # 3. Context Relevance
        context_relevance = analyse_context_relevance(model, tokenizer, context, question)
        num_extracted += context_relevance[0]
        num_sentences += context_relevance[1]

    # --- STEP 4: AGGREGATE SCORES ---
    # Prevent division by zero if lists are empty
    if len(statements) > 0:
        scores["faithfullness score"] = calculate_faithfulness_score(len(statements), supported_statements)
    else:
        scores["faithfullness score"] = 0
        
    if len(similatity) > 0:
        scores["answer relevance score"] = calculate_answer_relevance_score(similatity)
    else:
        scores["answer relevance score"] = 0
        
    scores["context relevance score"] = calculate_context_relevance_score(num_extracted, num_sentences)

    return scores

### Conclusion

Calculate scores for Qwen without temperature:

In [ ]:
#analyse_rag(model_qwen, tokenizer_qwen, do_sample=False)

Calculate scores for Qwen with temperature:

In [ ]:
#analyse_rag(model_qwen, tokenizer_qwen, do_sample=True)